In [1]:
import numpy as np
from operator import itemgetter

from tensorflow.keras.layers import Input
from yolov4.model import yolov4_body

In [2]:
def get_class(classes_path):
    with open(classes_path, encoding='utf-8') as f:
        classes = f.readlines()
    classes = [c.strip() for c in classes]
    return classes

def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def convert(input_shape, dn_weights_path, kr_weights_path, anchors_path, classes_path):
    anchors = get_anchors(anchors_path)
    classes = get_class(classes_path)
    num_anchors = len(anchors)
    num_classes = len(classes)

    # Load model, or construct model and load weights.
    yolo4_model = yolov4_body(Input(shape=input_shape), num_anchors//3, num_classes)

    # Read and convert darknet weight
    print('Loading weights...')
    weights_file = open(dn_weights_path, 'rb')
    major, minor, revision = np.ndarray(shape=(3, ), dtype='int32', buffer=weights_file.read(12))
    if (major*10+minor)>=2 and major<1000 and minor<1000:
        seen = np.ndarray(shape=(1,), dtype='int64', buffer=weights_file.read(8))
    else:
        seen = np.ndarray(shape=(1,), dtype='int32', buffer=weights_file.read(4))
    #print('Weights Header: ', major, minor, revision, seen)

    convs_to_load = []
    bns_to_load = []
    for i in range(len(yolo4_model.layers)):
        layer_name = yolo4_model.layers[i].name
        if layer_name.startswith('conv2d_'):
            convs_to_load.append((int(layer_name[7:]), i))
        if layer_name.startswith('batch_normalization_'):
            bns_to_load.append((int(layer_name[20:]), i))

    convs_sorted = sorted(convs_to_load, key=itemgetter(0))
    bns_sorted = sorted(bns_to_load, key=itemgetter(0))

    bn_index = 0
    for i in range(len(convs_sorted)):
        #print('Converting ', i)
        if i == 93 or i == 101 or i == 109:
            #no bn, with bias
            weights_shape = yolo4_model.layers[convs_sorted[i][1]].get_weights()[0].shape
            bias_shape = yolo4_model.layers[convs_sorted[i][1]].get_weights()[0].shape[3]
            filters = bias_shape
            size = weights_shape[0]
            darknet_w_shape = (filters, weights_shape[2], size, size)
            weights_size = np.product(weights_shape)

            conv_bias = np.ndarray(
                shape=(filters, ),
                dtype='float32',
                buffer=weights_file.read(filters * 4))
            conv_weights = np.ndarray(
                shape=darknet_w_shape,
                dtype='float32',
                buffer=weights_file.read(weights_size * 4))
            conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])
            yolo4_model.layers[convs_sorted[i][1]].set_weights([conv_weights, conv_bias])
        else:
            #with bn, no bias
            weights_shape = yolo4_model.layers[convs_sorted[i][1]].get_weights()[0].shape
            size = weights_shape[0]
            bn_shape = yolo4_model.layers[bns_sorted[bn_index][1]].get_weights()[0].shape
            filters = bn_shape[0]
            darknet_w_shape = (filters, weights_shape[2], size, size)
            weights_size = np.product(weights_shape)

            conv_bias = np.ndarray(
                shape=(filters, ),
                dtype='float32',
                buffer=weights_file.read(filters * 4))
            bn_weights = np.ndarray(
                shape=(3, filters),
                dtype='float32',
                buffer=weights_file.read(filters * 12))

            bn_weight_list = [
                bn_weights[0],  # scale gamma
                conv_bias,      # shift beta
                bn_weights[1],  # running mean
                bn_weights[2]   # running var
            ]
            yolo4_model.layers[bns_sorted[bn_index][1]].set_weights(bn_weight_list)

            conv_weights = np.ndarray(
                shape=darknet_w_shape,
                dtype='float32',
                buffer=weights_file.read(weights_size * 4))
            conv_weights = np.transpose(conv_weights, [2, 3, 1, 0])
            yolo4_model.layers[convs_sorted[i][1]].set_weights([conv_weights])

            bn_index += 1

    weights_file.close()
    yolo4_model.save_weights(kr_weights_path)
    print('Weights saved!')

In [4]:
input_shape = (608, 608, 3)
dn_weights_path = 'model_data/weights/yolov4-default.weights'
kr_weights_path = 'model_data/weights/yolov4-default_weights.h5'
anchors_path = 'model_data/anchors/yolov4_anchors.txt'
classes_path = 'model_data/classes/yolov4-default.names'

yolo4_model = convert(input_shape, dn_weights_path, kr_weights_path, anchors_path, classes_path)

Loading weights...
Weights saved!


In [4]:
input_shape = (608, 608, 3)
dn_weights_path = 'model_data/weights/yolov4-plate_type.weights'
kr_weights_path = 'model_data/weights/yolov4-plate_type_weights.h5'
anchors_path = 'model_data/anchors/yolov4_anchors.txt'
classes_path = 'model_data/classes/yolov4-plate_type.names'

yolo4_model = convert(input_shape, dn_weights_path, kr_weights_path, anchors_path, classes_path)

Loading weights...
Weights saved!


In [5]:
input_shape = (320, 320, 3)
dn_weights_path = 'model_data/weights/yolov4-plate_number.weights'
kr_weights_path = 'model_data/weights/yolov4-plate_number_weights.h5'
anchors_path = 'model_data/anchors/yolov4_anchors.txt'
classes_path = 'model_data/classes/yolov4-plate_number.names'

yolo4_model = convert(input_shape, dn_weights_path, kr_weights_path, anchors_path, classes_path)

Loading weights...
Weights saved!


In [4]:
input_shape = (416, 416, 3)
dn_weights_path = 'model_data/weights/yolov4-flower.weights'
kr_weights_path = 'model_data/weights/yolov4-flower_weights.h5'
anchors_path = 'model_data/anchors/yolov4_anchors.txt'
classes_path = 'model_data/classes/yolov4-flower.names'

yolo4_model = convert(input_shape, dn_weights_path, kr_weights_path, anchors_path, classes_path)

Loading weights...
Weights saved!
